In [1]:
import pandas as pd
import torch
import os

from torch.utils.data import Dataset
from transformers import (BertTokenizer, BertForSequenceClassification, Trainer,
                          TrainingArguments, BertPreTrainedModel)
# from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import accuracy_score, f1_score


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [3]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert'
TRAIN_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/CogniSenti/twitter_fbPost_merged_train.tsv'
TEST_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/CogniSenti/twitter_fbPost_merged_test.tsv'
EVAL_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/CogniSenti/twitter_fbPost_merged_dev.tsv'


In [4]:
def tsv_to_text(tsv_file_loc):
    file_name = tsv_file_loc.split("/")[-1].split(".")[0]
    txt_name = tsv_file_loc.replace(".tsv", ".txt")
    txt_name = txt_name.replace("split_merged", "texts")

    if os.path.exists(txt_name):
        return [txt_name, file_name]

    df = pd.read_csv(tsv_file_loc, sep="\t")

    for txt in df["text"]:
        with open(txt_name, "a", encoding="utf8") as f:
            f.writelines(txt + "\n")
    return [txt_name, file_name]


tsv_to_text(TRAIN_FILE_LOC)


['/home/abdullah/Code/dl/bnlp-resources/sentiment/CogniSenti/twitter_fbPost_merged_train.txt',
 'twitter_fbPost_merged_train']

In [5]:
def tsv_to_df(csv_file_loc):
    df = pd.read_csv(csv_file_loc, sep='\t')
    # remove id column
    df = df.drop(columns=['id'])

    # replace neutral with 0, positive with 1, negative with 2 in class_label column
    df.loc[df['class_label'] == 'Neutral', 'class_label'] = 0
    df.loc[df['class_label'] == 'Positive', 'class_label'] = 1
    df.loc[df['class_label'] == 'Negative', 'class_label'] = 2

    return df


train_df = tsv_to_df(TEST_FILE_LOC)

texts = train_df['text'].tolist()
print(train_df)


                                                  text class_label
0    ফ্রাঁন্সের সেই শক্তী-ধর রাজা লুই-কেত্রোজ ( রাজ...           2
1    ৯৯% রাই ভুল করে এবার আপনাদের পালা!!! #আইকিউ_টে...           0
2    ঢাকা শহরের হাফ ভাগ মানুষ বাকি আড়াই ভাগ অমানুষ...           2
3    ছোট বেলায় দেখেছি চোর কে গলায় জুতার মালা পড়ি...           2
4    ..ভাব যদি কোথায় সে ঘাসের আশ্রয়ে চলে গেল - ভা...           0
..                                                 ...         ...
981  বিভিন্ন দেশের মুদ্রার নাম ➺ বাংলাদেশ – টাকা। ➺...           0
982  @Im_acs আমি ৮ বছর ধরে প্রবাসে আছি এখনো জাই নি ...           2
983  ১৭৫৭সন ২৩সে জুন পলাশী থেকে ১৬ ই ডিসেম্বর /১৯৭১...           0
984  সবুজ পাহাড়ে নজরকাড়া ফুল। পানখাইয়া পাড়া, খাগড়াছ...           1
985  ধা‌র কর‌তে কর‌তে জীবন টা ম‌নে হয় শেষ হ‌য়ে যা...           2

[986 rows x 2 columns]


In [6]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


In [7]:
def f1_calculator(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  f1 = f1_score(labels, preds, average='weighted')
  return {
      'f1': f1,
  }


In [8]:
train_list = tsv_to_text(TRAIN_FILE_LOC)
test_list = tsv_to_text(TEST_FILE_LOC)
eval_list = tsv_to_text(EVAL_FILE_LOC)


In [9]:
os.environ["WANDB_DISABLED"] = "true"

train_df = tsv_to_df(TRAIN_FILE_LOC)
test_df = tsv_to_df(TEST_FILE_LOC)
eval_df = tsv_to_df(EVAL_FILE_LOC)


class TINYDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_df['text'].tolist(
), padding=True, truncation=True, max_length=512)

train_dataset = TINYDataset(train_encodings, train_df['class_label'].tolist())
test_dataset = TINYDataset(test_encodings, test_df['class_label'].tolist())
eval_dataset = TINYDataset(eval_encodings, eval_df['class_label'].tolist())

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14


Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'bert.encoder.layer.0.attention.output.dense.weight_orig', 'cls.predictions.bias', 'bert.encoder.layer.1.output.dense.weight_orig', 'bert.encoder.layer.0.attention.output.dense.weight_mask', 'bert.encoder.layer.0.attention.self.query.weight_orig', 'bert.encoder.layer.0.attention.self.value.weight_orig', 'bert.encoder.layer.1.attention.self.value.weight_orig', 'bert.encoder.layer.0.attention.self.value.weight_mask', 'bert.encoder.layer.0.attention.self.query.weight_mask', 'bert.encoder.layer.1.attention.self.key.weight_mask', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.1.attention.self.key.weight_orig', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.dense.weig

## 1 Epoch Train

In [10]:
training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=1,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=96,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=96,
    save_steps=96,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset

)

trainer.train()


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 4599
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 96
100%|██████████| 96/96 [00:05<00:00, 18.73it/s]***** Running Evaluation *****
  Num examples = 985
  Batch size = 96


{'loss': 1.0819, 'learning_rate': 9.600000000000001e-06, 'epoch': 1.0}


                                               
100%|██████████| 96/96 [00:05<00:00, 18.73it/s]Saving model checkpoint to temp/checkpoint-96
Configuration saved in temp/checkpoint-96/config.json


{'eval_loss': 1.061453938484192, 'eval_runtime': 0.3256, 'eval_samples_per_second': 3025.646, 'eval_steps_per_second': 33.789, 'epoch': 1.0}


Model weights saved in temp/checkpoint-96/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-96 (score: 1.061453938484192).
100%|██████████| 96/96 [00:07<00:00, 12.42it/s]

{'train_runtime': 7.7365, 'train_samples_per_second': 594.455, 'train_steps_per_second': 12.409, 'train_loss': 1.0818875630696614, 'epoch': 1.0}


TrainOutput(global_step=96, training_loss=1.0818875630696614, metrics={'train_runtime': 7.7365, 'train_samples_per_second': 594.455, 'train_steps_per_second': 12.409, 'train_loss': 1.0818875630696614, 'epoch': 1.0})

## 100 Epoch with 1 epoch model

In [12]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/1/1_cogni_sentiment_50'
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14

for name, param in model.named_parameters():
    if 'bert.encoder.layer.0' in name:
        # print(name, param.requires_grad)
        param.requires_grad = False
# print("\n\n\n")
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=400,
    save_steps=400,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=f1_calculator,

)

trainer.train()

# print(cool.metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()


loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/1/1_cogni_sentiment_50/config.json
Model config BertConfig {
  "_name_or_path": "/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vo

{'loss': 0.9977, 'learning_rate': 4e-05, 'epoch': 16.67}


                                                  
 17%|█▋        | 400/2400 [01:12<05:55,  5.62it/s]Saving model checkpoint to temp/checkpoint-400
Configuration saved in temp/checkpoint-400/config.json


{'eval_loss': 0.9121167063713074, 'eval_f1': 0.4037269048734925, 'eval_runtime': 0.3818, 'eval_samples_per_second': 2580.188, 'eval_steps_per_second': 7.858, 'epoch': 16.67}


Model weights saved in temp/checkpoint-400/pytorch_model.bin
 33%|███▎      | 800/2400 [02:24<04:45,  5.60it/s]***** Running Evaluation *****
  Num examples = 985
  Batch size = 384


...


100%|██████████| 2400/2400 [07:15<00:00,  5.74it/s]Saving model checkpoint to temp/checkpoint-2400
Configuration saved in temp/checkpoint-2400/config.json


{'eval_loss': 1.0013792514801025, 'eval_accuracy': 0.6233502538071066, 'eval_runtime': 0.3756, 'eval_samples_per_second': 2622.397, 'eval_steps_per_second': 7.987, 'epoch': 100.0}


Model weights saved in temp/checkpoint-2400/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-400 (score: 0.8230096697807312).
100%|██████████| 2400/2400 [07:16<00:00,  5.50it/s]

{'train_runtime': 436.1689, 'train_samples_per_second': 1054.408, 'train_steps_per_second': 5.502, 'train_loss': 0.7117468833923339, 'epoch': 100.0}


TrainOutput(global_step=2400, training_loss=0.7117468833923339, metrics={'train_runtime': 436.1689, 'train_samples_per_second': 1054.408, 'train_steps_per_second': 5.502, 'train_loss': 0.7117468833923339, 'epoch': 100.0})